In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
#from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary
import torch.nn.functional as F


/home/MMI24limbyungwan/.conda/envs/byungwan_resn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

print(os.environ.get('CUDA_VISIBLE_DEVICES'))

cuda
3


In [3]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

In [4]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        directory_path = './data/224'
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path)
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = np.round(mask).astype(np.uint8)
        mask[mask > 12] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

# class CustomDataset_target(Dataset):
#     def __init__(self, csv_file, transform=None, infer=False):
#         self.data = pd.read_csv(csv_file)
#         self.transform = transform
#         self.infer = infer

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         directory_path = "/mnt/nas27/Dataset/Samsung_DM"
#         img_path = self.data.iloc[idx, 1]
#         img_path = os.path.join(directory_path, img_path[2:])
#         image = cv2.imread(img_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#         if self.infer:
#             if self.transform:
#                 image = self.transform(image=image)['image']
#             return image


#         if self.transform:
#             augmented = self.transform(image=image)
#             image = augmented['image']
            

#         return image
     

transform = A.Compose(
    [   
        #A.Resize(224, 224),
        #A.Resize(128, 128),
        A.Normalize(),
        
        # 변형
        # A.VerticalFlip(p=0.5),
        # A.RandomRotate90(p=0.5),
        # A.HueSaturationValue(p=0.2),
        
        ToTensorV2()
    ]
)

In [5]:
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(self, x):
        return x.view_as(x)
    @staticmethod
    def backward(self, grad_output): # 역전파 시에 gradient에 음수를 취함
        return grad_output * (-1)

class domain_classifier(nn.Module):
    def __init__(self):
        super(domain_classifier, self).__init__()
        self.fc1 = nn.Linear(224*224*64, 10)
        self.fc2 = nn.Linear(10, 4) # source = 0, target = 1 회귀 가정

    def forward(self, x):
        x = x.view(-1, 224*224*64)
        x = GradReverse.apply(x) # gradient reverse
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        
        #return torch.sigmoid(x)
        return x

class domain_linear(nn.Module):
    def __init__(self):
        super(domain_linear, self).__init__()
        self.fc1 = nn.Linear(224*224*64, 10)
        self.fc2 = nn.Linear(10, 1) # source = 0, target = 1 회귀 가정

    def forward(self, x):
        x = x.view(-1, 224*224*64)
        x = GradReverse.apply(x) # gradient reverse
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        
        return torch.sigmoid(x)
        #return x
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(IdentityBlock, self).__init__()
        
        # 3x3 convolution
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU()
        
        # 3x3 convolution
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()
        
        # Skip connection
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Adding the skip connection
        out += self.skip(identity)
        out = self.relu2(out)
        
        return out
#인코더 블럭
class Conv2(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv2,self).__init__() 
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        p = self.maxpool(x)
        
        return x , p
class Conv3(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv3,self).__init__()         
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,in_channels)
        self.identityblock4 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        x = self.identityblock4(x)
        p = self.maxpool(x)
        
        return x , p
class Conv4(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv4,self).__init__()         
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,in_channels)
        self.identityblock4 = IdentityBlock(in_channels,in_channels)
        self.identityblock5 = IdentityBlock(in_channels,in_channels)
        self.identityblock6 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        x = self.identityblock4(x)
        x = self.identityblock5(x)
        x = self.identityblock6(x)
        p = self.maxpool(x)
        
        return x , p
class Conv5(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv5,self).__init__() 
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
        
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        p = self.maxpool(x)
        
        return x , p
#디코더 블럭
class DecoderBlock(nn.Module):
    def __init__(self, channels):
        super(DecoderBlock, self).__init__()
        self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1) # output_padding 추가
        self.convblock1 = IdentityBlock(channels*2, channels)

    def forward(self, x, skip):
        x = self.upsample(x)
        if x.size(2) != skip.size(2) or x.size(3) != skip.size(3):
            x = F.interpolate(x, size=(skip.size(2), skip.size(3)))
        x = torch.cat([x, skip], dim=1)
        x = self.convblock1(x)
        #print("x",x.shape,"skip: ",skip.shape)
        return x

#Unet구조 middle의 xm값의 움직임에 주의
class Resnet34_Unet(nn.Module):
    def __init__(self,n_classes):
        super(Resnet34_Unet,self).__init__()
        self.fconv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.fbn1 = nn.BatchNorm2d(64)
        self.frelu1 = nn.ReLU()
        self.fmaxpooling = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        
        self.conv2 = Conv2(64,128)
        self.conv3 = Conv3(128,256)
        self.conv4 = Conv4(256,512)
        self.conv5 = Conv5(512,1024)
        
        self.middleconv = IdentityBlock(1024,2048)
        self.dropout = nn.Dropout2d(0.1) #
           
        self.decoder5 = DecoderBlock(1024)
        self.decoder4 = DecoderBlock(512)
        self.decoder3 = DecoderBlock(256)
        self.decoder2 = DecoderBlock(128)
        self.decoder1 = DecoderBlock(64)
        self.transpose = nn.ConvTranspose2d(64, 64, kernel_size=4, stride=2, padding=1) # output_padding 추가
        
        self.segmap = nn.Conv2d(64,n_classes, kernel_size=1)
        self.domain_classifier = domain_classifier()
        self.domain_linear = domain_linear()
        
    def forward(self,x):
        x = self.fconv1(x)#3->64
        x0 = self.fbn1(x)
        x1 = self.frelu1(x)
        p = self.fmaxpooling(x1)#첫 conv: x0([8, 64, 109, 109]) p([8, 64, 54, 54])
        #print("conv1: ",x1.shape, "maxpooling: ",p.shape)
        x2,p = self.conv2(p)
        #print("conv2: ",x2.shape, "maxpooling: ",p.shape)
        x3,p = self.conv3(p)
        #print("conv3: ",x3.shape, "maxpooling: ",p.shape)
        x4,p = self.conv4(p)
        #print("conv4: ",x4.shape, "maxpooling: ",p.shape)
        x5,p = self.conv5(p)
        #print("conv5: ",x5.shape, "maxpooling: ",p.shape)
        
        xm = self.middleconv(p)#xm([8, 4096, 2, 2])
        #print("xm: ",xm.shape, "maxpooling: ",p.shape)
        xm = self.dropout(xm)
        
        x = self.decoder5(xm,x5)#뉴런:2048*2->2048 1
        x = self.decoder4(x,x4)#뉴런:1024*2->1024 
        x = self.decoder3(x,x3) #14
        x = self.decoder2(x,x2)#28
        x = self.decoder1(x,x1)#55
        x = self.transpose(x)
        
        #print(x.shape)
        #x = F.interpolate(x, size=(224, 224))
        x_c = self.segmap(x)
        x_d = self.domain_linear(x)
        
        return x_c,x_d

In [6]:
# import pickle

# # 저장된 class_weights를 불러옵니다.
# class_weights_path = 'CLASS_WEIGHTS.pkl'

# with open(class_weights_path, 'rb') as file:
#     CLASS_WEIGHTS = pickle.load(file)

# print(CLASS_WEIGHTS)

## Loss Function

In [7]:
# loss function과 optimizer 정의

class DANN_Loss(nn.Module):
    def __init__(self):
        super(DANN_Loss, self).__init__()

        self.CE = nn.CrossEntropyLoss() # 0~9 class 분류용
        self.BCE = nn.BCELoss() # 도메인 분류용
        
    # result : DANN_CNN에서 반환된 값
    # label : 숫자 0 ~ 9에 대한 라벨
    # domain_num : 0(source) or 1(target)
    def forward(self, result, label, domain_num, alpha = 1):
        label_logits, domain_logits = result # DANN_CNN의 결과

        batch_size = domain_logits.shape[0]

        domain_target = torch.FloatTensor([domain_num] * batch_size).unsqueeze(1).to(device)

        domain_loss = self.BCE(domain_logits, domain_target) # domain 분류 loss
        target_loss = self.CE(label_logits, label) # class 분류 loss

        loss = target_loss + alpha * domain_loss

        return loss
    

loss_fn = DANN_Loss().to(device)

#criterion =nn.CrossEntropyLoss()
#domain_criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss(weight=class_weights)

In [8]:
LR = 0.001
EP = 20
BATCH_SIZE = 16
ACCMULATION_STEP = 1 
N_CLASSES = 13 #IoU 점수측정하기 위한 클래스의 개수
ALPHA = 0.5
# model 초기화
#model = Unet_resnet18(n_classes = N_CLASSES).to(device)
#model = ResNet50(num_classes=N_CLASSES).to(device)
model = Resnet34_Unet(n_classes = N_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

optimizer.zero_grad() 

source_dataset = CustomDataset(csv_file='./data/DA_csv/train_source.csv', transform=transform)
source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True)
target_dataset = CustomDataset(csv_file='./data/DA_csv/fish_train_source.csv', transform=transform)
target_dataloader = DataLoader(target_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_source_dataset = CustomDataset(csv_file='./data/DA_csv/val_source.csv', transform=transform)
val_source_dataloader = DataLoader(val_source_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_target_dataset = CustomDataset(csv_file='./data/DA_csv/fish_val_source.csv', transform=transform)
val_target_dataloader = DataLoader(val_target_dataset, batch_size=BATCH_SIZE, shuffle=False)
# source_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_source.csv'), transform=transform)#
# source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
# s_valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform)
# s_valid_dataloader = DataLoader(s_valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
# #target_dataset = CustomDataset_target(csv_file='./f_train_source.csv', transform=transform)
# target_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./fish_val_source.csv'), transform=transform)
# target_dataloader = DataLoader(target_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

# t_test_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./test.csv'), transform=transform, infer=True)
# t_test_dataloader = DataLoader(t_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

In [9]:
# import pandas as pd
# import os
# import cv2
# import matplotlib.pyplot as plt
# a1 = pd.read_csv("./data/6_fish_source.csv")
# p = "./data/224/"
# a2 = os.path.join(p, a1.iloc[3,2])
# a3 = cv2.imread(a2)
# a4 = cv2.cvtColor(a3, cv2.COLOR_BGR2GRAY)
# a4 = np.round(a4).astype(np.uint8)
# a5 = a4*20

# plt.imshow(a5, cmap='gray')
# plt.axis('off')
# plt.show()

In [10]:
import random
#torch.cuda.empty_cache()
# import wandb


# wandb.init(
#     # set the wandb project where this run will be logged
#     project="practice_10_19",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR,
#     "architecture": "CNN",
#     "dataset": "Samsung",
#     "epochs": EP,
#     }
# )

for epoch in range(EP):
    # 클래스별 IoU를 누적할 리스트 초기화
    train_class_ious = []
    fish_train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0

    for source_data, target_data in tqdm(zip(source_dataloader, target_dataloader)):

        source_images = source_data[0].float().to(device)
        source_masks = source_data[1].long().to(device)
        target_images = target_data[0].float().to(device)
        target_masks = target_data[1].long().to(device)

        optimizer.zero_grad()
        source_outputs = model(source_images)
        target_outputs = model(target_images)

        source_loss = loss_fn(source_outputs, source_masks, 0, alpha = ALPHA)
        target_loss = loss_fn(target_outputs, target_masks, 1, alpha = ALPHA)

        loss = source_loss + target_loss

        epoch_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #scheduler.step()
                # train 클래스별 IoU 계산
        source_outputs = torch.softmax(source_outputs[0], dim=1).cpu()
        source_outputs = torch.argmax(source_outputs, dim=1).numpy()

        for class_id in range(N_CLASSES):
            iou = calculate_iou_per_class(np.array(source_masks.cpu()), np.array(source_outputs), class_id)
            train_class_ious.append(iou)

        target_outputs = torch.softmax(target_outputs[0], dim=1).cpu()
        target_outputs = torch.argmax(target_outputs, dim=1).numpy()

        for class_id in range(N_CLASSES):
            iou = calculate_iou_per_class(np.array(target_masks.cpu()), np.array(target_outputs), class_id)
            fish_train_class_ious.append(iou)

    train_class_ious = np.array(train_class_ious).reshape(N_CLASSES, -1)
    train_class_ious = np.mean(train_class_ious, axis=1)
    print("--IoU Scores Train--")
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    fish_train_class_ious = np.array(fish_train_class_ious).reshape(N_CLASSES, -1)
    fish_train_class_ious = np.mean(fish_train_class_ious, axis=1)
    print()
    print("--IoU Scores Fish Train--")
    for class_id, iou in enumerate(fish_train_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    train_mIoU = np.mean(train_class_ious)
    fish_train_mIoU = np.mean(fish_train_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Train Loss: {(epoch_loss/len(source_dataloader))}")
    print(f"Train mIoU: {train_mIoU}, Fish_Train_mIoU: {fish_train_mIoU}" )
    print("___________________________________________________________________________________________\n")


    ################################################################
    # 클래스별 IoU를 누적할 리스트 초기화
    val_class_ious = []
    fish_val_class_ious = []
    epoch_loss = 0
    # 학습
    with torch.no_grad():
        model.eval()

        for source_data, target_data in tqdm(zip(val_source_dataloader, val_target_dataloader)):

            source_images = source_data[0].float().to(device)
            source_masks = source_data[1].long().to(device)
            target_images = target_data[0].float().to(device)
            target_masks = target_data[1].long().to(device)

            source_outputs = model(source_images)
            target_outputs = model(target_images)

            source_loss = loss_fn(source_outputs, source_masks, 0, alpha = ALPHA)
            target_loss = loss_fn(target_outputs, target_masks, 1, alpha = ALPHA)

            loss = source_loss + target_loss

            epoch_loss += loss.item()

            # train 클래스별 IoU 계산
            source_outputs = torch.softmax(source_outputs[0], dim=1).cpu()
            source_outputs = torch.argmax(source_outputs, dim=1).numpy()

            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(source_masks.cpu()), np.array(source_outputs), class_id)
                val_class_ious.append(iou)

            target_outputs = torch.softmax(target_outputs[0], dim=1).cpu()
            target_outputs = torch.argmax(target_outputs, dim=1).numpy()

            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(target_masks.cpu()), np.array(target_outputs), class_id)
                fish_val_class_ious.append(iou)

    val_class_ious = np.array(val_class_ious).reshape(N_CLASSES, -1)
    val_class_ious = np.mean(val_class_ious, axis=1)
    print("--IoU Scores val--")
    for class_id, iou in enumerate(val_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    fish_val_class_ious = np.array(fish_val_class_ious).reshape(N_CLASSES, -1)
    fish_val_class_ious = np.mean(fish_val_class_ious, axis=1)
    print()
    print("--IoU Scores Fish val--")
    for class_id, iou in enumerate(fish_val_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    val_mIoU = np.mean(val_class_ious)
    fish_val_mIoU = np.mean(fish_val_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"val Loss: {(epoch_loss/len(val_source_dataloader))}")
    print(f"val mIoU: {val_mIoU}, Fish_val_mIoU: {fish_val_mIoU}" )
    print("___________________________________________________________________________________________\n")

#     # log metrics to wandb
#     wandb.log({"train score": train_mIoU, "train score2": fish_train_mIoU})
#     wandb.log({"val score": val_mIoU, "val score2": fish_val_mIoU})
    
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()


69it [01:03,  1.08it/s]


--IoU Scores Train--
Class0: 0.0198 Class1: 0.0485 Class2: 0.1192 Class3: 0.1930 Class4: 0.2376 Class5: 0.2605 Class6: 0.2725 
Class7: 0.2628 Class8: 0.2854 Class9: 0.3214 Class10: 0.3054 Class11: 0.3211 Class12: 0.3276 
--IoU Scores Fish Train--
Class0: 0.0842 Class1: 0.1039 Class2: 0.1481 Class3: 0.1906 Class4: 0.2183 Class5: 0.2545 Class6: 0.2755 
Class7: 0.2657 Class8: 0.2848 Class9: 0.3154 Class10: 0.3006 Class11: 0.3154 Class12: 0.3196 
Epoch1
Train Loss: 50.039515688799426
Train mIoU: 0.22883813882116838, Fish_Train_mIoU: 0.23666371214297313
___________________________________________________________________________________________



15it [00:07,  2.01it/s]


--IoU Scores val--
Class0: 0.2641 Class1: 0.2759 Class2: 0.2357 Class3: 0.2681 Class4: 0.3003 Class5: 0.2611 Class6: 0.3497 
Class7: 0.2770 Class8: 0.2424 Class9: 0.2485 Class10: 0.3335 Class11: 0.2595 Class12: 0.3348 
--IoU Scores Fish val--
Class0: 0.1190 Class1: 0.1093 Class2: 0.0969 Class3: 0.1035 Class4: 0.1342 Class5: 0.1241 Class6: 0.1172 
Class7: 0.1062 Class8: 0.1030 Class9: 0.1023 Class10: 0.1776 Class11: 0.1097 Class12: 0.1217 
Epoch1
val Loss: 53.26100463867188
val mIoU: 0.28083072923083086, Fish_val_mIoU: 0.11728254275264434
___________________________________________________________________________________________



69it [01:03,  1.09it/s]


...

--IoU Scores val--
Class0: 0.2598 Class1: 0.2971 Class2: 0.2490 Class3: 0.2962 Class4: 0.2821 Class5: 0.3079 Class6: 0.3477 
Class7: 0.2569 Class8: 0.2662 Class9: 0.2712 Class10: 0.3418 Class11: 0.2564 Class12: 0.3460 
--IoU Scores Fish val--
Class0: 0.3441 Class1: 0.3285 Class2: 0.2791 Class3: 0.2828 Class4: 0.3200 Class5: 0.3239 Class6: 0.4063 
Class7: 0.3005 Class8: 0.2992 Class9: 0.2700 Class10: 0.3788 Class11: 0.2875 Class12: 0.3675 
Epoch19
val Loss: 51.514044189453124
val mIoU: 0.2906443660578557, Fish_val_mIoU: 0.32216171677479283
___________________________________________________________________________________________



69it [01:03,  1.09it/s]


--IoU Scores Train--
Class0: 0.5480 Class1: 0.5219 Class2: 0.5138 Class3: 0.5238 Class4: 0.5169 Class5: 0.5472 Class6: 0.5810 
Class7: 0.5487 Class8: 0.5640 Class9: 0.5787 Class10: 0.5512 Class11: 0.5980 Class12: 0.5784 
--IoU Scores Fish Train--
Class0: 0.4469 Class1: 0.4189 Class2: 0.4249 Class3: 0.4399 Class4: 0.3997 Class5: 0.4316 Class6: 0.4681 
Class7: 0.4357 Class8: 0.4576 Class9: 0.4690 Class10: 0.4477 Class11: 0.4673 Class12: 0.4732 
Epoch20
Train Loss: 50.40164405712183
Train mIoU: 0.5516605577502226, Fish_Train_mIoU: 0.4446522853608592
___________________________________________________________________________________________



15it [00:07,  2.01it/s]

--IoU Scores val--
Class0: 0.3375 Class1: 0.3624 Class2: 0.2984 Class3: 0.3697 Class4: 0.3782 Class5: 0.3625 Class6: 0.4385 
Class7: 0.3383 Class8: 0.3056 Class9: 0.3212 Class10: 0.4251 Class11: 0.3210 Class12: 0.4437 
--IoU Scores Fish val--
Class0: 0.2862 Class1: 0.2745 Class2: 0.2400 Class3: 0.2537 Class4: 0.2946 Class5: 0.2852 Class6: 0.3075 
Class7: 0.2524 Class8: 0.2490 Class9: 0.2397 Class10: 0.3336 Class11: 0.2492 Class12: 0.2836 
Epoch20
val Loss: 51.25402272542318
val mIoU: 0.3616982696694216, Fish_val_mIoU: 0.27301794925658945
___________________________________________________________________________________________



In [12]:
torch.save(model.state_dict(), './data/DA_resnet34_1106.pth')

In [11]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'